In [1]:
from KeRLas import Brain, TimedGymEnv, Trainer
from KeRLas.models import defaultQModel
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from cartpole import CartPoleEnv

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(CartPoleEnv(), tlimit=300)

space = env.observation_space
high = np.array(
    [
        2.4,
        1.0,
        12 * 2 * math.pi / 360,
        1.0
    ]
)
env.RandomObservationSpace = gym.spaces.Box(-high, high)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n

rlmodel = LateralDiffModel(defaultQModel(observation_width, nactions), 0.9)
policy = BoltzmannQPolicy(0.0001)
brain = Brain(rlmodel, policy, training_policies=map(BoltzmannQPolicy, [10.0, 1.0, 0.1, 0.01, 0.001]))

mbsize = 50
#for i in range(10):
#    next(brain.Memory.generate_samples(20))
#_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [ ]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record):
        env.render()

trainer = Trainer(env, brain, 0.5, 10000)   

from KeRLas import GymPlayer
p = GymPlayer(env, brain, callback=Callback())

#temps = [10.0, 1.0, 0.1, 0.01, 0.001]

for i in xrange(20):
    #training_tau = temps[i%len(temps)]
    #policy.tau = training_tau
    #brain.training_model().fit_generator(
    #    brain.trainig_data_generator(mbsize),
    #    steps_per_epoch = 1000, epochs = 10, verbose = True, workers = 0
    #)
    
    metrics = trainer.train(mbsize, 1000, 10)
    
    nsum = 0
    N = 5
    #policy.tau = 0.0001
    with brain.training(False):
        for _ in range(N):
            history = p.runEpisode()
            #print len(history)
            nsum += len(history)
        print float(nsum)/N, metrics

12.4 0.0022033998
36.4 0.017816909
33.4 0.013669387
21.6 0.013696946
8.2 0.0033747768
21.4 0.016248548


In [ ]:
for _ in xrange(100):
    p.runEpisode()